In [4]:
import numpy as np


In [3]:
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# file = open("chat.txt").read()

KeyboardInterrupt: ignored

In [5]:
file = open("chat.txt").read()
len(file)
import random
import sys


In [6]:
chars = sorted(list(set(file)))
len(chars)

469

In [7]:
char_indices = dict((c,i) for i, c in enumerate(chars))
indices_char = dict((i,c) for i,c in enumerate(chars))

In [8]:
maxlen = 40
step = 3
sentences = []
next_chars = []
file = file[1000000:]

for i in range(0, len(file) - maxlen, step):
    sentences.append(file[i: i + maxlen])
    next_chars.append(file[i + maxlen])
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
import tensorflow as tf
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [10]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(file) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = file[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=on_epoch_end)

In [11]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [12]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [13]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)


Epoch 1/5
4070/4070 [==============================] - 826s 202ms/step - loss: 1.5287

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "attached file scanned image 
too much on"
attached file scanned image 
too much on the all some be the that is because what is the the can that is the that was the that me the that is the lies and dad the the to the that what and have the the read the to people the see that to the that is the to the to the that is the that is the see that is the that is and that is the good
19/12/2020, 22:34 - +91 94802 54643: I don't the let's and it the the the to the that was the make that w
----- diversity: 0.5
----- Generating with seed: "attached file scanned image 
too much on"
attached file scanned image 
too much on a selled to are that is much it like the turn
20/12/2020, 01:19 - +91 89712 76807: I can wait
10/12/2020, 22:39 - +91 94802 54643: I have it in read you want in to no how no put the send that what is the thatar

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


8:14 - +91 96861 84307: You will the who real started to play the say is it im the for that and the playing the compent a one you can sear will they the bost the sall the hatting the this sext of the the and play the on the for the one a shiting man of the pu loting the the to you 
----- diversity: 1.0
----- Generating with seed: "2/2020, 02:10 - Ishaan: so whenever some"
2/2020, 02:10 - Ishaan: so whenever some and dinerated you to play hamis i good of in into you really dogem ctrets
28/12/2020, 21:17 - Ishaan: +1
17/12/2020, 19:19 - Murali Unfortunately: <Media omitted>
13/01/2021, 29:13 - Anoop K. N: Sof of utky viet was a all of people
11/01/2021, 14:4o - +91 96861 84307: <Media omitted>
20/12/2020, 09:10 - Ishaan: em conten
12/12/2020, 22:15 - Murali Unfortunately: _?
08/01/2021, 19:59 - Murali Unfo
----- diversity: 1.2
----- Generating with seed: "2/2020, 02:10 - Ishaan: so whenever some"
2/2020, 02:10 - Ishaan: so whenever some gro
20/01/2021, 07:28 - Ishaan: SsIf
08/01/2021, 18

In [16]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(file) - maxlen - 1)
    generated = ''
    sentence = file[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [17]:
print(generate_text(10000, 0.5))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


020, 14:14 - +91 89712 76807: Sad
17/12/2020, 15:24 - +91 96201 79084: That is the game interes
08/01/2021, 19:08 - +91 96201 79084: Yeah I want to play and back from the man no
16/01/2021, 15:34 - Sachxn: I am it is this the play this mush are send the play as you agreating that
07/01/2021, 19:12 - +91 96201 79084: Which who beain thing
11/12/2020, 15:17 - Pareekshith: ah
08/01/2021, 23:16 - Sush: Shreasion this was make shit shit
06/01/2021, 13:52 - +91 96201 79084: <Media omitted>
28/12/2020, 15:18 - Sachxn: <Media omitted>
23/12/2020, 12:13 - +91 96201 79084: <Media omitted>
24/12/2020, 20:30 - +91 96201 79084: <Media omitted>
06/01/2021, 23:12 - +91 96201 79084: The spam man was the coming the don't should laugh why
20/01/2021, 14:47 - +91 96201 79084: That make the pirly there was some with the famor and see the enting this this me
21/01/2021, 15:32 - Sush: You want to so better sam the see that of the succ
08/01/2021, 23:57 - Sush: there is mart weach and the paran the sid
21/12